In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/SERSTRUT/features'

Mounted at /content/drive
/content/drive/My Drive/SERSTRUT/features


In [ ]:
import numpy as np
import pandas as pd
import sqlite3
from tqdm import tqdm

class Preprocessing:


  def import_data(self):
    conn = sqlite3.connect('office4.db')
    cur_1 = conn.cursor()

    conn = sqlite3.connect('h355-2015.sqlite3')
    cur_2 = conn.cursor()
    return cur_1, cur_2

  def loading_feature_office_1(self, row, cur_1):

    cur_1.execute("SELECT * FROM "+ row)
    rows = cur_1.fetchall()
    data = pd.DataFrame(data = rows)
    data.columns = ['time', 'value', 'date']
    data = data.sort_values('date')
    return data


  def loading_class_office_1(self, cur_1):
    cur_1.execute("SELECT * FROM 'labels'")
    rows = cur_1.fetchall()
    data = pd.DataFrame(data = rows)
    data.columns = ['date', 'RMS', 'number_occupants', 'time', 'value']
    data = data.sort_values('date')
    return data


  def create_feature_office_1(self, x):
    new_time = int(list(time_value.keys())[0]) + (x*1000)
    #print("new_time", new_time)
    #print("len data:",len(data))
    for i in tqdm(range(len(data))):

      if (new_time >= int(list(time_value.keys())[i])) and (new_time < int(list(time_value.keys())[i+1])):
        res = float(list(time_value.values())[i])
        #print("res", res)
        break
    return res

  def data_processing_office_1(self):
    global time_value
    global data

    columns = ['co2','RMS', 'power1', 'power2', 'power3', 'power4', 'power5',
       'power6', 'OCCUPANCY', 'corridor', 'Temperature', 'OUTDOOR_TEMPERATURE',
       'Door_contact', 'window', 'labels']

    Final_Data = pd.DataFrame(data = np.arange(0,3600*24*10,600), columns = ['time'])

    cur_1, cur_2 = self.import_data()
    for i in columns:
      if i == 'labels':
        data = self.loading_class_office_1(cur_1)
        time_value = {data.iloc[j,3]:data.iloc[j,2] for j in range(len(data))}
      else:
        data = self.loading_feature_office_1(i, cur_1)
        time_value = {data.iloc[j,0]:data.iloc[j,1] for j in range(len(data))}

      create_feature = self.create_feature_office_1
      Final_Data[i] = Final_Data['time'].apply(create_feature)

    Final_Data.labels = Final_Data.labels.round()
    return Final_Data

  def loading_feature_office_2(self, cur_2):

    cur_2.execute("SELECT * From 'data'")
    rows = cur_2.fetchall()
    data1 = pd.DataFrame(data = rows)
    data2 = pd.read_csv("rms.csv", header = None)
    data = pd.concat([data1, data2])
    data.columns = ['variableidref','time', 'value', 'date']
    data = data.sort_values('time')

    data = data[pd.to_datetime(data.date) >= pd.to_datetime('14/03/2016 04:53:16')]
    data = data[pd.to_datetime(data.date) <= pd.to_datetime('21/03/2016 15:00:16')]

    return data

  def loading_class_office_2(self):
    data = pd.read_csv("labels_H355.csv")
    data.columns = ['time', 'value', 'date']
    return data

  def create_feature_office_2(self,x):
    new_time = int(list(time_value.keys())[0]) + (x*1000)
    for i in tqdm(range(len(data))):
      try:

        if (new_time >= int(list(time_value.keys())[i])) and (new_time < int(list(time_value.keys())[i+1])):
          res = list(time_value.values())[i]
          break
      except: res = list(time_value.values())[i]
    return res

  def create_class_office_2(self,x):
    new_time = pd.to_datetime('14/03/2016 04:53:16') + pd.to_timedelta(x, unit='s')

    for i in range(len(data)):
      try:
        res = 0
        if (new_time >= pd.to_datetime(list(time_value.keys())[i])) and (new_time < pd.to_datetime(list(time_value.keys())[i+1])):
          res = list(time_value.values())[i]
          break
      except : res = list(time_value.values())[i]
    return res


  def data_processing_office_2(self):
    global time_value
    global data

    columns = {'power1':5,'co2':15, 'RMS':30, 'power2':11, 'power3':14, 'power4': 23, 'Occupancy':19, 'Temperature': 17, 'Temperature1': 10, 'Temperature12': 7, 'door1': 13, 'window':12}
    #'RMS':30,'power1':5,  'power2':11, 'power3':14, 'power4': 23, 'Occupancy':19, 'window':12
    #'power1':5,'co2':15, 'RMS':30, 'power2':11, 'power3':14, 'power4': 23, 'Occupancy':19, 'Temperature': 17, 'Temperature1': 10, 'Temperature12': 7, 'door1': 13, 'window':12
    Final_Data = pd.DataFrame(data = np.arange(0,3600*24*7 + 10*3600 ,600), columns = ['time'])

    cur_1, cur_2 = self.import_data()
    new_data = self.loading_feature_office_2(cur_2)
    for j in columns:
      data = new_data[new_data.variableidref == columns[j]]
      time_value = {data.iloc[i,1]:data.iloc[i,2] for i in range(len(data))}
      print("time_value: ", time_value)
      create_feature = self.create_feature_office_2
      Final_Data[j] = Final_Data.time.apply(create_feature)

    data = self.loading_class_office_2()
    time_value = {data.iloc[i,2]:data.iloc[i,1] for i in range(len(data))}
    transform_label = self.create_class_office_2
    Final_Data['labels'] = Final_Data.time.apply(transform_label)

    return Final_Data




#script

preprocessing = Preprocessing()
#Final_Data_S1 = preprocessing.data_processing_office_1()
#Final_Data_S1.to_csv("H358_new_edited_NM_2.csv")

Final_Data_S2 = preprocessing.data_processing_office_2()
#Final_Data_S2.to_csv("H355_debugging_2.csv")

time_value:  {1457928374: 0.006557, 1457930177: 0.006728, 1457931980: 0.007375, 1457933783: 0.010247, 1457935586: 0.016664, 1457937389: 0.018973, 1457939192: 0.025038, 1457940995: 0.023941, 1457942799: 0.013297, 1457944604: 0.024903, 1457946407: 0.017279, 1457948211: 0.013946, 1457950014: 0.02312, 1457951817: 0.023343, 1457953620: 0.009925, 1457955425: 0.011011, 1457957229: 0.014403, 1457959031: 0.007647, 1457960834: 0.02163, 1457962636: 0.012408, 1457964439: 0.024523, 1457966242: 0.011209, 1457968045: 0.018044, 1457969847: 0.012861, 1457971651: 0.01038, 1457973457: 0.015652, 1457975264: 0.025379, 1457977067: 0.032415, 1457978872: 0.021025, 1457980676: 0.026832, 1457982480: 0.013425, 1457984283: 0.015629, 1457986088: 0.006354, 1457987891: 0.005951, 1457989694: 0.005914, 1457991497: 0.005689, 1457993300: 0.005421, 1457995103: 0.00557, 1457996906: 0.005337, 1457998709: 0.005377, 1458000512: 0.005401, 1458002315: 0.005294, 1458004118: 0.005346, 1458005921: 0.005423, 1458007724: 0.00525, 1

100%|██████████| 284/284 [00:00<00:00, 100904.90it/s]


time_value:  {1457927870729: 22.0, 1457928473913: 22.0, 1457929077199: 22.0, 1457929680559: 22.0, 1457930283854: 22.0, 1457930887138: 22.0, 1457931490372: 22.0, 1457932093595: 22.0, 1457932696910: 22.0, 1457933300296: 22.0, 1457933903607: 22.0, 1457934506824: 22.0, 1457935110060: 22.0, 1457935713277: 22.0, 1457936316498: 22.0, 1457936919882: 22.0, 1457937523354: 22.0, 1457938126703: 22.0, 1457938730042: 22.0, 1457939333384: 22.0, 1457939936704: 22.0, 1457940540029: 22.0, 1457941143298: 22.0, 1457941746505: 22.0, 1457942349774: 22.0, 1457942953032: 22.0, 1457943556417: 22.0, 1457944159760: 22.0, 1457944763194: 22.0, 1457945366488: 22.0, 1457945969768: 22.0, 1457946573188: 22.0, 1457947176614: 22.0, 1457947560853: 27.0, 1457947684343: 46.0, 1457947704623: 51.0, 1457947725689: 85.0, 1457947746789: 105.0, 1457947767867: 80.0, 1457947792769: 87.0, 1457947813049: 81.0, 1457947883125: 85.0, 1457947913241: 80.0, 1457947933517: 86.0, 1457947988527: 81.0, 1457948015634: 93.0, 1457948054791: 106.

100%|█████████▉| 3951/3953 [00:00<00:00, 14123.50it/s]


time_value:  {1457928126654: 0.0, 1457928730854: 0.0, 1457929335025: 0.0, 1457929939141: 0.0, 1457930543367: 0.0, 1457931147687: 0.0, 1457931751902: 0.0, 1457932356075: 0.0, 1457932960340: 0.0, 1457933564558: 0.0, 1457934168866: 0.0, 1457934773172: 0.0, 1457935377426: 0.0, 1457935981758: 0.0, 1457936586052: 0.0, 1457937190334: 0.0, 1457937794649: 0.0, 1457938398865: 0.0, 1457939003095: 0.0, 1457939607408: 0.0, 1457940211805: 0.0, 1457940816098: 0.0, 1457941420266: 0.0, 1457942024446: 0.0, 1457942628637: 0.0, 1457943232880: 0.0, 1457943837085: 0.0, 1457944441260: 0.0, 1457945045443: 0.0, 1457945649628: 0.0, 1457946253771: 0.0, 1457946857957: 0.0, 1457946918674: 44.0, 1457946939396: 55.0, 1457946960511: 58.0, 1457946982023: 54.0, 1457947002730: 61.0, 1457947024242: 58.0, 1457947054397: 61.0, 1457947084556: 11.0, 1457947105262: 30.0, 1457947126371: 67.0, 1457947147478: 62.0, 1457947172005: 68.0, 1457947310704: 64.0, 1457947331399: 68.0, 1457947935582: 70.0, 1457948390361: 66.0, 1457948869

...

100%|█████████▉| 1794/1798 [00:00<00:00, 18889.75it/s]


time_value:  {1457935794911: 1.0, 1457936392734: 0.0, 1457937588223: 0.0, 1457938752830: 1.0, 1457938872568: 1.0, 1457939023672: 1.0, 1457939154288: 1.0, 1457939297993: 1.0, 1457939419881: 1.0, 1457939540391: 1.0, 1457939668073: 1.0, 1457939792312: 1.0, 1457939913906: 1.0, 1457940069028: 1.0, 1457940189143: 1.0, 1457940333394: 1.0, 1457940454043: 1.0, 1457940581417: 1.0, 1457940702436: 1.0, 1457940822380: 1.0, 1457940948047: 1.0, 1457941084234: 1.0, 1457941204596: 1.0, 1457941332595: 1.0, 1457941482048: 1.0, 1457941609229: 1.0, 1457941811850: 1.0, 1457941946893: 1.0, 1457942067055: 1.0, 1457942198923: 1.0, 1457942318666: 1.0, 1457942438828: 1.0, 1457942559214: 1.0, 1457943004200: 1.0, 1457943140990: 1.0, 1457943261382: 1.0, 1457943400290: 1.0, 1457943563976: 1.0, 1457943684944: 1.0, 1457943811296: 1.0, 1457943937572: 1.0, 1457944058444: 1.0, 1457944183748: 1.0, 1457944310439: 1.0, 1457944442182: 1.0, 1457944561911: 1.0, 1457944682427: 1.0, 1457944803922: 1.0, 1457944935491: 1.0, 145794

100%|██████████| 1470/1470 [00:00<00:00, 24601.30it/s]


time_value:  {1457928808664: 0.0, 1457930354519: 0.0, 1457931603323: 0.0, 1457933149706: 0.0, 1457934696295: 0.0, 1457936004994: 0.0, 1457937313827: 0.0, 1457938682319: 0.0, 1457940050898: 0.0, 1457941717172: 0.0, 1457943264771: 0.0, 1457944634117: 0.0, 1457945825147: 0.0, 1457947374051: 0.0, 1457948923402: 0.0, 1457950115710: 0.0, 1457951785392: 0.0, 1457953157124: 0.0, 1457954588969: 0.0, 1457956200167: 0.0, 1457957811765: 0.0, 1457959483281: 0.0, 1457961154982: 0.0, 1457962587752: 0.0, 1457963901219: 0.0, 1457965095393: 0.0, 1457966707717: 0.0, 1457968319932: 0.0, 1457969573821: 0.0, 1457971185764: 0.0, 1457972737736: 0.0, 1457973991026: 0.0, 1457975184609: 0.0, 1457976617169: 0.0, 1457977870433: 0.0, 1457979541011: 0.0, 1457981032195: 0.0, 1457982344159: 0.0, 1457983655922: 0.0, 1457984907765: 0.0, 1457986218855: 0.0, 1457987529655: 0.0, 1457988720986: 0.0, 1457989912024: 0.0, 1457991162325: 0.0, 1457992829167: 0.0, 1457994436150: 0.0, 1457995804895: 0.0, 1457997411511: 0.0, 145799

100%|██████████| 431/431 [00:00<00:00, 75335.27it/s]


In [ ]:
Final_Data_S2.to_csv("H355_NM_To_add_columns.csv")

In [ ]:
Final_Data_S2

,time,power1,labels
0,0,22.0,0
1,600,22.0,0
2,1200,22.0,0
3,1800,22.0,0
4,2400,22.0,0
...,...,...,...
1063,637800,23.0,1
1064,638400,23.0,3
1065,639000,24.0,3
1066,639600,23.0,3


In [ ]:
Final_Data_S2.to_csv("H355_debugging.csv")

In [ ]:
Final_Data_S2 = preprocessing.data_processing_office_2()

IndexError: list index out of range

In [ ]:
list(time_value.keys())[0]

IndexError: list index out of range

In [ ]:
conn = sqlite3.connect('h355-2015.sqlite3')
cur_2 = conn.cursor()
cur_2.execute("SELECT * From 'data'")
rows = cur_2.fetchall()
data1 = pd.DataFrame(data = rows)
data2 = pd.read_csv("rms.csv", header =None)
#data = pd.concat([data1, data2])

In [ ]:
time_value

{}

In [ ]:
data2

,0,1,2,3
0,30,1457695785,0.021721,11/03/2016 12:29:45
1,30,1457697595,0.019389,11/03/2016 12:59:55
2,30,1457699401,0.034258,11/03/2016 13:30:01
3,30,1457701209,0.023639,11/03/2016 14:00:09
4,30,1457703015,0.015269,11/03/2016 14:30:15
...,...,...,...,...
902,30,1459453997,0.006510,31/03/2016 21:53:17
903,30,1459455854,0.006640,31/03/2016 22:24:14
904,30,1459457705,0.005842,31/03/2016 22:55:05
905,30,1459459561,0.005877,31/03/2016 23:26:01


In [ ]:
data = pd.concat([data1, data2])

In [ ]:
data

,0,1,2,3
0,0,1420676817200,1.000000,08/01/2015 01:26:57
1,0,1420965687035,1.000000,11/01/2015 09:41:27
2,0,1420966163918,1.000000,11/01/2015 09:49:23
3,0,1420966544234,1.000000,11/01/2015 09:55:44
4,0,1420967051235,1.000000,11/01/2015 10:04:11
...,...,...,...,...
902,30,1459453997,0.006510,31/03/2016 21:53:17
903,30,1459455854,0.006640,31/03/2016 22:24:14
904,30,1459457705,0.005842,31/03/2016 22:55:05
905,30,1459459561,0.005877,31/03/2016 23:26:01


In [ ]:
df1 = pd.DataFrame(
    {
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
        "C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
    },
    index=[0, 1, 2, 3],
)




In [ ]:
df2 = pd.DataFrame(
    {
        "A": ["A4", "A5", "A6", "A7"],
        "B": ["B4", "B5", "B6", "B7"],
        "C": ["C4", "C5", "C6", "C7"],
        "D": ["D4", "D5", "D6", "D7"],
    },
    index=[0, 1, 2, 3],
)

In [ ]:
frames = [df1, df2]

result = pd.concat(frames)

In [ ]:
result

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
0,A4,B4,C4,D4
1,A5,B5,C5,D5
2,A6,B6,C6,D6
3,A7,B7,C7,D7


In [ ]:
Final_Data_S2.shape
Final_Data_S1.shape

NameError: name 'Final_Data_S2' is not defined

In [ ]:
Final_Data_S2.head()

In [ ]:
Final_Data_S1.head()